## Start

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from numpy import mean
from numpy import std
from numpy import cov
import pandas as pd
import matplotlib.pyplot as plt
import scipy.linalg as la
from scipy.stats import pearsonr

directory='your well log data input.las'
data=np.loadtxt(directory,skiprows=136)

header=['DEPTH','GR','CALI','SP','MSFL','LLD','LLS','DT','DTL','RHOB','NPHI','PEF','DRHO']
datapd=pd.DataFrame(data,columns=header)
data_raw=datapd[['DEPTH','GR','CALI','SP','MSFL','LLD','LLS','DT','DTL','RHOB','NPHI','PEF','DRHO']]
data_raw.set_index('DEPTH')

data_raw_all_np=datapd[['DEPTH','DT','RHOB','NPHI']]
data_pca_np_all=data_raw_all_np.to_numpy()
data_pca_np_all[data_pca_np_all==-999.2500]=np.nan
data_pca_np_all=data_pca_np_all[~np.isnan(data_pca_np_all).any(axis=1)]
np.set_printoptions(suppress=True)
header=['DEPTH','DT','RHOB','NPHI']
list_data_non_null=pd.DataFrame(data_pca_np_all,columns=header)
pd.set_option("display.max_rows", None, "display.max_columns", None)

data_chosen_depth=list_data_non_null.loc[794:3019]
data_raw_used=data_chosen_depth[['DEPTH','DT','RHOB','NPHI']]
data_pca_np=data_raw_used.to_numpy()
data_pca_np[data_pca_np==-999.2500]=np.nan
data_pca_np=data_pca_np[~np.isnan(data_pca_np).any(axis=1)]
np.set_printoptions(suppress=True)
depth=np.reshape(data_chosen_depth[['DEPTH']].to_numpy(),(len(data_chosen_depth[['DEPTH']]),1))
dt=np.reshape(data_pca_np[:,1],(len(data_pca_np),1))
rhob=np.reshape(data_pca_np[:,2],(len(data_pca_np),1))
nphi=np.reshape(data_pca_np[:,3],(len(data_pca_np),1))

data_pca_np=data_pca_np[:,1:4]
norm=np.linalg.norm(data_pca_np)
normal_array=data_pca_np/norm

header=['norm_DT','norm_RHOB','norm_NPHI']
norm_pd=pd.DataFrame(normal_array,columns=header)

corr_coef_pd=norm_pd.corr(method='pearson')
corr_coef_np=corr_coef_pd.to_numpy()

eigen=la.eig(corr_coef_np)
result_eigenvalue=eigen[0]
result_eigenvector=eigen[1]

eigvals,eigvecs=la.eig(corr_coef_np)
eigvals=eigvals.real
lambda1=eigvals[0]
lambda2=eigvals[1]
lambda3=eigvals[2]

W1=lambda1/np.sum(lambda1+lambda2+lambda3)
W2=lambda2/np.sum(lambda1+lambda2+lambda3)
W3=lambda3/np.sum(lambda1+lambda2+lambda3)

v1=eigvecs[:,0].reshape(3,1)

corr_coef_np@v1

lambda1*v1

L_index=1.09785553*(dt)-1.35333656*(rhob)+1.30352311*(nphi)

pca_L_index_np_stack=np.column_stack((depth,dt,rhob,nphi,L_index))
header=['DEPTH','DT','RHOB','NPHI','L-INDEX']
pca_L_index_pd=pd.DataFrame(pca_L_index_np_stack,columns=header)

coal=[]
for i in rhob:
    if 1.1<i<1.4:
        coal.append(1)
    else:
        coal.append(0)
coal=np.array(coal)

graph,(plot1,plot2,plot3,plot4,plot5)=plt.subplots(1,5)
plt.rcParams['figure.figsize']=(8,8)

plot1.plot(dt,depth,color='red')
plot1.set_title('DT')
plot1.yaxis.set_visible(True)

plot2.plot(rhob,depth,color='blue')
plot2.set_title('RHOB')
plot2.yaxis.set_visible(False)

plot3.plot(nphi,depth,color='green')
plot3.set_title('NPHI')
plot3.yaxis.set_visible(False)

plot4.plot(L_index,depth,color='olive')
plot4.set_title('PCA')
plot4.yaxis.set_visible(False)

plot5.plot(coal,depth,color='black')
plot5.set_title('Coal')
plot5.yaxis.set_visible(False)

## End